<a href="https://colab.research.google.com/github/sufiyansayyed19/LLM_Learning/blob/main/Day5_1_CV_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:
Using Portfolio web page link find the relevent links, scan them and using the porfolio page and other link pages content generate an attractive cv.

All imports  
import os  
import json  
from IPython.display import Markdown, display, update_display  
from google.colab import userdata  
from openai import OpenAI   

## 1.Scraper Functions

This section defines functions to scrape website content and extract links. These functions are crucial for gathering the necessary information to create the brochure.

### Get website content

In [1]:
# 1. Install necessary libraries
!pip install beautifulsoup4 requests markdownify

import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin

# --- THESE ARE THE FUNCTIONS ED HAS IN 'scraper.py' ---

def fetch_website_contents(url):
    """
    Fetches the text content of a website, stripping out scripts and styles.
    """
    print(f"Scraping: {url}...")
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
        response = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Remove script and style elements
        for script in soup(["script", "style", "nav", "footer"]):
            script.extract()

        # Get text
        text = soup.get_text()

        # Break into lines and remove leading/trailing space on each
        lines = (line.strip() for line in text.splitlines())
        # Break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        # Drop blank lines
        text = '\n'.join(chunk for chunk in chunks if chunk)

        return text[:5000] # Limit to 5000 chars to save tokens
    except Exception as e:
        return f"Error fetching {url}: {e}"



### Get website links

In [2]:
def fetch_website_links(url):
    """
    Fetches all links from a website and converts relative links to absolute.
    """
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
        response = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')

        links = []
        for a_tag in soup.find_all('a', href=True):
            href = a_tag['href']
            # Convert relative links (e.g. "/about") to full links
            full_url = urljoin(url, href)
            if full_url.startswith('http'):
                links.append(full_url)

        # Remove duplicates
        return list(set(links))
    except Exception as e:
        print(f"Error fetching links: {e}")
        return []

print("Scraper functions loaded!")

Scraper functions loaded!


## 2.API Key Set Up for colab

### ADD key value in secret and check with code below

In [3]:
from google.colab import userdata

try:
    key = userdata.get('OPENAI_API_KEY')
    print(f"Success! Key found. It starts with: {key[:8]}...")
except Exception as e:
    print("Error: Could not find key. Did you turn the toggle switch ON?")

Success! Key found. It starts with: sk-or-v1...


## 3.API Call Setup

In [4]:
import os
from openai import OpenAI

 #1. Setup API Key (Make sure you added OPENAI_API_KEY in Colab Secrets)
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# 2. Setup Client (OpenRouter)
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ["OPENAI_API_KEY"],
)

# 3. Define Models
# We use a cheap, fast model for link selection
LINK_MODEL = "openai/gpt-4o-mini"
# We use a smarter model for writing the brochure
WRITER_MODEL = "openai/gpt-4o-mini"

print("Client and Models configured!")

Client and Models configured!


## 4.Prompt Design

### i-System Prompts

#### Link system prompt

In [13]:
# --- SYSTEM PROMPTS ---
link_system_prompt = """
You are provided with a list of links extracted from a candidate’s portfolio website.

Your task is to intelligently decide **which links are most relevant** for generating a
**high-impact, one-page professional CV**.

Prioritize links that contain information about:
- About / Profile / Summary
- Projects or Case Studies
- Skills or Technologies
- Experience, Internships, Freelance work
- Education or Certifications
- Blogs, Research, or Open-source (if technical)

Ignore or de-prioritize links that are:
- Redundant navigation pages
- Social media without technical content
- Contact-only pages
- Generic landing or marketing pages with no personal details

You will be given the links in the following JSON format:
{
    "links": [
        {"type": "About Me", "url": "https://full.url/goes/here/about"},
        {"type": "Projects", "url": "https://another.full.url/projects"},
        {"type": "Contact", "url": "https://another.full.url/contact"}
    ]
}

Select only the links that are **most useful for building a strong, credible, and recruiter-ready CV**.
"""


#### Brochure System prompt

In [14]:

cv_system_prompt = """
You are an expert career assistant and resume writer.

You will be provided with a portfolio website URL that contains multiple internal links (such as About, Projects, Experience, Skills, Blogs, GitHub, Case Studies, etc.).

Your task is to:

Analyze the entire portfolio, including all relevant linked pages

Extract meaningful information about:

Skills and technologies

Projects and their impact

Experience, internships, or freelancing work

Education and certifications

Open-source contributions or research (if any)

Achievements and measurable results

Infer strengths, specialization, and career direction based on evidence (not assumptions)

Then generate a high-quality, ATS-friendly, and recruiter-attractive CV tailored for modern tech roles (e.g., Software Engineer, ML Engineer, Data Scientist, AI Engineer, Full-Stack Developer — depending on the portfolio content).

CV Requirements

Use strong action verbs and impact-driven bullet points

Quantify results wherever possible (performance, scale, users, accuracy, efficiency, etc.)

Highlight real projects over generic skills

Reflect industry-level professionalism, not student-level wording

Optimize for clarity, conciseness, and credibility

Output Format (in Markdown)

Professional Summary (2–3 lines, sharp and role-focused)

Core Skills (grouped by category)

Experience / Internships / Freelance (if applicable)

Projects (most important section)

Education

Certifications / Achievements (if available)

Optional: Publications, Blogs, Open Source

Do not invent information.
If something is missing, omit it gracefully.
Focus on making the candidate look hire-ready and competitive.
"""


### ii- User Prompt

#### User prompt to select relevent links

In [21]:
def get_links_user_prompt(url):
    raw_links = fetch_website_links(url)

    # Take only first 30 links to save tokens
    links_text = "\n".join(raw_links[:30])

    user_prompt = f"""
You are analyzing a **portfolio website** for the purpose of generating a
**high-impact, one-page professional CV**.

Below is a list of links found on the website:
{url}

Your task is to:
- Identify ONLY the links that are **useful for building a strong CV**
- Prioritize links related to:
  - About / Profile / Bio
  - Projects / Case Studies
  - Skills / Technologies
  - Experience / Internships / Freelance
  - Education / Certifications
  - Blogs, Research, Open Source (if technical)

Exclude links that are:
- Terms of Service, Privacy Policy
- Contact-only pages
- Email or mailto links
- Social media links with no technical content
- Generic navigation or marketing pages

### Output Requirements
- Respond ONLY in **valid JSON**
- Return a list of **full HTTPS URLs**
- Do NOT include explanations or extra text

### Example Output Format
{{
  "relevant_links": [
    "https://example.com/about",
    "https://example.com/projects",
    "https://example.com/experience"
  ]
}}

Links:
{links_text}
"""
    return user_prompt


#### User prompt to Create brochure

In [22]:
def get_cv_user_prompt(candidate_name, portfolio_url):
    print("📥 Fetching portfolio content (this may take a moment)...")
    content = fetch_page_and_all_relevant_links(portfolio_url)

    user_prompt = f"""
You are an expert career assistant and professional resume writer.

You are analyzing the **portfolio website** of a candidate named: {candidate_name}

The portfolio contains multiple linked pages such as About, Projects, Skills, Experience,
Blogs, GitHub, Case Studies, or similar sections.

Your task is to:
- Analyze the entire portfolio content provided below
- Extract verified information about:
  - Technical skills and tools
  - Projects and their real-world impact
  - Internships, freelance work, or professional experience
  - Education and certifications
  - Open-source contributions, research, or blogs (if present)
- Infer strengths and specialization ONLY from the given evidence

Then generate a **high-quality, ATS-friendly, recruiter-ready CV** in **markdown format**
(without code blocks).

### CV Writing Guidelines
- Use strong action verbs and impact-focused bullet points
- Quantify results where possible (accuracy, performance, scale, users, revenue, etc.)
- Emphasize real projects over generic skill lists
- Maintain industry-level, professional language
- Do NOT invent or assume missing information

### CV Structure
- Professional Summary (2–3 concise lines)
- Core Skills (grouped by category)
- Experience / Internships / Freelance (if available)
- Projects (most important section)
- Education
- Certifications / Achievements (if available)
- Optional: Publications, Blogs, Open Source

Here is the portfolio content to analyze:

{content[:10000]}
"""

    # Limited to 10k characters to avoid context overflow
    return user_prompt


## 5.Content Get functions

### Page and all Links function

In [23]:
def fetch_page_and_all_relevant_links(url):
    # 1. Get Main Page
    contents = fetch_website_contents(url)

    # 2. Get Relevant Links
    relevant_links = select_relevant_links(url)

    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"

    # 3. Get Content of Relevant Links
    for link in relevant_links['links']:
        print(f"   Reading: {link['type']} ({link['url']})")
        link_content = fetch_website_contents(link["url"])
        result += f"\n\n### Link: {link['type']}\n{link_content}"

    return result

### Select relevant links from all links

In [24]:
import json

def select_relevant_links(url):
    print(f"🔍 Analyzing links for {url}...")
    response = client.chat.completions.create(
        model=LINK_MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    try:
        links = json.loads(result)
        print(f"✅ Found {len(links['links'])} relevant links.")
        return links
    except:
        print("Error parsing JSON")
        return {"links": []}

### 6.Brochure Generator function

In [25]:
from IPython.display import Markdown, display, update_display

def stream_cv(candidate_name, portfolio_url):
    print(f"🚀 Generating high-impact CV for {candidate_name}...")

    stream = client.chat.completions.create(
        model=WRITER_MODEL,
        messages=[
            {"role": "system", "content": cv_system_prompt},
            {"role": "user", "content": get_cv_user_prompt(candidate_name, portfolio_url)}
        ],
        stream=True
    )

    response = ""
    display_handle = display(
        Markdown("⏳ Analyzing portfolio and crafting CV..."),
        display_id=True
    )

    for chunk in stream:
        content = chunk.choices[0].delta.content or ""
        response += content
        update_display(
            Markdown(response),
            display_id=display_handle.display_id
        )


## 7.Driver

In [26]:
# --- RUN IT! ---
# Use standard HuggingFace URL
stream_cv("Sufiyan Portfolio Site", "https://sufiyan-sayyed.vercel.app/")

🚀 Generating high-impact CV for Sufiyan Portfolio Site...
📥 Fetching portfolio content (this may take a moment)...
Scraping: https://sufiyan-sayyed.vercel.app/...
🔍 Analyzing links for https://sufiyan-sayyed.vercel.app/...
Error parsing JSON


**Sufiyan Sayyed**  
[Your Email] | [Your Phone Number] | [LinkedIn Profile] | [GitHub Profile]  

---

### Professional Summary
Detail-oriented Full-Stack Developer with a strong foundation in Artificial Intelligence and Data Science. Proven expertise in building scalable, high-performance web applications using modern technologies. Committed to delivering exceptional user experiences and optimizing digital solutions for business growth.

### Core Skills
#### Programming Languages
- JavaScript, Python, Java, SQL, HTML5, CSS3

#### Frontend Technologies
- React.js, Zustand, Next.js, Vite

#### Backend Technologies
- Node.js, Express.js

#### Database Management
- MongoDB, PostgreSQL, Cloudinary

#### Other Skills
- AWS, Git, Socket.io, JWT, HTTP-Only Cookies

### Experience
**Freelance Web Developer**  
*Various Clients* - January 2024 - Present  
- Designed and implemented custom web solutions catering to diverse business needs, enhancing client satisfaction through strategic problem-solving.

### Projects
**Cartix E-Commerce Platform**  
*Technologies*: MERN Stack  
- Developed a scalable architecture with role-based access, enabling secure user authentication and real-time cart updates.
- Improved user engagement by 30% through enhanced UX design and streamlined order tracking.

**Likup Chat Application**  
*Technologies*: React, Socket.io  
- Created an instant messaging application with secure authentication and end-to-end state management.
- Achieved message delivery performance of under 100ms, enhancing real-time communication.

**SmartCanteen System**  
*Technologies*: React, Zustand  
- Implemented complex state management and role-based routing to efficiently handle user interactions.
- Optimized performance using React 18 & Vite, resulting in faster load times and improved user satisfaction.

**Lumina Skincare Web App**  
*Technologies*: Next.js, Custom Animations  
- Built a visually appealing, SEO-optimized portfolio for a dermatology practice, showcasing services and testimonials.
- Increased site traffic by 20% within the first month post-launch through enhanced SEO tactics.

**TechAsia IoT Products Platform**  
*Technologies*: React, Express API  
- Developed a decoupled frontend with robust content control and secure media management, facilitating efficient customer engagement.

### Education
**BE in Artificial Intelligence & Data Science**  
Vivekanand Education Society’s Institute of Technology, Mumbai - Aug 2023 – June 2026  
CGPA: 9.54 (till Sem 6)

**Diploma in Electrical Engineering**  
M.H. Saboo Siddik Polytechnic, Byculla, Mumbai - Dec 2020 – Jul 2023  
Percentage: 89.5%

### Certifications / Achievements
- Data Analyst Bootcamp - Udemy, 2025
- Fundamentals of Deep Learning - NVIDIA, 2024
- Complete Full-Stack Web Development Bootcamp - Udemy, 2024
- Introduction to Front-end Development - Coursera (Meta), 2024
- Programming in Python - Coursera (Meta), July 2024

### Optional: Publications, Blogs, Open Source
**Open Source Contributions**  
- Actively contributing to various projects on GitHub, focusing on improving existing features and collaborative development practices.  

---

*Note: Customize your email, phone number, and any relevant links before sending or printing this CV.*